## Import packages and data

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

#import lightgbm as lgb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import sys

In [3]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import tensorflow as tf
from tensorflow import keras

In [ ]:
df_item=pd.read_csv('meta-item.csv')
df_store=pd.read_csv('meta-store.csv')
df_sales=pd.read_csv('sales.csv')

df_submission=pd.read_csv('submission_sample.csv')
df_submission[['DATE', 'S100', 'I100', 'C100', 'C101']] = df_submission['ID'].str.split('_',expand=True)
df_submission['S100']=df_submission['S100'].astype(int)
df_submission['I100']=df_submission['I100'].astype(int)

In [4]:
def preprocess(df):
    
    df[['year', 'month', 'day']] = df['DATE'].str.split('-',expand=True)
    
    for value in [ 'S100', 'I100', 'C100', 'C101','month', 'year', 'day']:        
        df[value] = df[value].astype(str).astype(int)
    
    df = df.merge(df_item,on=['I100'])
    df = df.merge(df_store,on=['S100'])
    

    df['DATE'] = pd.to_datetime(df['DATE']).dt.strftime('%Y-%m-%d')    
    return df

In [5]:
df_sales= preprocess(df_sales)
df_submission =preprocess(df_submission)
df_submission.head(5)

,ID,QTT,DATE,S100,I100,C100,C101,year,month,day,I101,I102,I103,S101,S102,S103
0,2021-10-03_0_1_12_140,-1,2021-10-03,0,1,12,140,2021,10,3,2,1,1,1,17,10
1,2021-10-03_0_1_12_164,-1,2021-10-03,0,1,12,164,2021,10,3,2,1,1,1,17,10
2,2021-10-03_0_1_12_339,-1,2021-10-03,0,1,12,339,2021,10,3,2,1,1,1,17,10
3,2021-10-03_0_1_13_128,-1,2021-10-03,0,1,13,128,2021,10,3,2,1,1,1,17,10
4,2021-10-03_0_1_14_164,-1,2021-10-03,0,1,14,164,2021,10,3,2,1,1,1,17,10


## Divide dataset

### Divide X and Y

In [6]:
features_names =['S100', 'I100', 'C100', 'C101', 'month', 'year', 'day', 'I101', 'I102', 'I103', 'S101', 'S102', 'S103']
features_names_qtt =['QTT', 'S100', 'I100', 'C100', 'C101', 'month', 'year', 'day', 'I101', 'I102', 'I103', 'S101', 'S102', 'S103']
X_lstm=df_sales[features_names_qtt]
#Training
X = df_sales[features_names]
Y = df_sales['QTT']

#Submission
x_sub= df_submission[features_names]
y_sub=df_submission['QTT']

In [7]:
#Scale train data 
from sklearn.preprocessing import MinMaxScaler

scaler=MinMaxScaler()
data_scaled= scaler.fit_transform(X_lstm)
features = data_scaled
Y_scaled = data_scaled[:,0]

In [8]:
Y_scaled.shape

(2749369,)

In [9]:
#Scale submission Data
submission_scaled= scaler.fit_transform(x_sub)
sub_features = submission_scaled
y_sub = submission_scaled[:,0]

### For LGBM

In [10]:
from sklearn.model_selection import TimeSeriesSplit

x_train, x_test, y_train,y_test = train_test_split(X,Y, test_size=0.2, random_state=42)  
X_validation, X_test, y_validation, y_test=train_test_split(x_test, y_test, test_size=0.5, random_state=42)

### For LSTM

In [10]:
sequence_len = 32
batch_size= 64
features_len = 14

X_train, X_test, y_train, y_test = train_test_split(features, Y_scaled, test_size=0.2, random_state=42)  
X_valid= sub_features
y_valid=y_sub

train_generator = TimeseriesGenerator(X_train, y_train, length=sequence_len, batch_size=batch_size)
test_generator= TimeseriesGenerator(X_test, y_test, length=sequence_len, batch_size=batch_size)
valid_generator= TimeseriesGenerator(X_valid, y_valid, length=sequence_len, batch_size=batch_size)

# LGBM

In [12]:
hyper_params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': ['rmse'],
    'learning_rate': 0.1,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.7,
    'bagging_freq': 10,
    "max_depth": 8,
    "num_leaves": 64,  
    "max_bin": 512,
    "num_iterations": 100
}

In [13]:
gbm = lgb.LGBMRegressor(**hyper_params)

In [ ]:
gbm.fit(x_train, y_train, eval_set=[(X_validation, y_validation)], setval_metric='rmse', early_stopping_rounds=30, verbose=50)

# LSTM

In [11]:
features, Y_scaled =train_generator[0]

print(len(train_generator))
print("batch x shape:", X_lstm.shape)
print('batch y shape:', Y.shape)
print(train_generator[0])

34367
batch x shape: (2749369, 14)
batch y shape: (2749369,)
(array([[[0.        , 0.2       , 0.21052632, ..., 1.        ,
         0.47368421, 0.7       ],
        [0.        , 1.        , 0.86315789, ..., 1.        ,
         0.        , 1.        ],
        [0.00144928, 0.7       , 0.52631579, ..., 1.        ,
         0.57894737, 0.8       ],
        ...,
        [0.        , 0.8       , 0.2       , ..., 1.        ,
         0.68421053, 0.6       ],
        [0.00144928, 0.46666667, 0.49473684, ..., 1.        ,
         0.15789474, 0.2       ],
        [0.        , 0.66666667, 0.11578947, ..., 1.        ,
         0.78947368, 1.        ]],

       [[0.        , 1.        , 0.86315789, ..., 1.        ,
         0.        , 1.        ],
        [0.00144928, 0.7       , 0.52631579, ..., 1.        ,
         0.57894737, 0.8       ],
        [0.        , 0.66666667, 0.18947368, ..., 1.        ,
         0.78947368, 1.        ],
        ...,
        [0.00144928, 0.46666667, 0.49473684, .

In [12]:
model = keras.models.Sequential()
model.add(keras.layers.LSTM(32, input_shape=(sequence_len, features_len), return_sequences=True))
model.add(tf.keras.layers.LeakyReLU(alpha=0.5))
model.add(keras.layers.LSTM(32, return_sequences=True))
model.add(tf.keras.layers.LeakyReLU(alpha=0.5))
model.add(tf.keras.layers.Dropout(0.5))
model.add(keras.layers.LSTM(32, return_sequences=False))
model.add(tf.keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(1))

model.summary()

2022-02-02 22:52:48.068053: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2022-02-02 22:52:48.068449: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 12. Tune using inter_op_parallelism_threads for best performance.


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 32, 32)            6016      
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 32, 32)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 32, 32)            8320      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 32, 32)            0         
_________________________________________________________________
dropout (Dropout)            (None, 32, 32)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0

In [13]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=2,
                                                    mode='min')

model.compile(loss=tf.losses.MeanSquaredError(),
              optimizer=tf.optimizers.Adam(),
              metrics=[tf.metrics.MeanAbsoluteError()])


Simply to long to train - that's why the error

In [14]:
history = model.fit_generator(train_generator, epochs=50,
                    validation_data=test_generator,
                    shuffle=False,
                    callbacks=[early_stopping])

Epoch 1/50
 2316/34367 [=>............................] - ETA: 128:11:00 - loss: 5.7977e-05 - mean_absolute_error: 0.0038

KeyboardInterrupt: 

In [ ]:
saver=tf.train.Saver()
save_path='checkpoints/'
#while training you can store using
saver.save(sess=session,save_path=save_path)
#and restore
saver.restore(sess=session,save_path=save_path)

# Load Model

In [ ]:
loaded_model = tf.keras.models.load_model(f'{run_dir}/best_model.h5')

In [ ]:
model.evaluate_generator(test_generator, verbose=0)  

In [ ]:
#Predict on test set
predictions_test=model.predict_generator(test_generator)

### Test on submission

In [ ]:
#Predict on test set
predictions_submission=model.predict_generator(valid_generator)

In [ ]:
# Concat my pred with the actual 
df_pred = pd.concat([pd.DataFrame(predictions), pd.DataFrame(x_test[:,1:][sequence_len:])], axis=1)

In [ ]:
df_pred

In [ ]:
#Inverse the scaling
rev_trans=scaler.inverse_transform(df_pred) 
rev_trans

Too long to train